In [ ]:
!python --version

Python 3.8.16


In [18]:
!pip freeze

absl-py==1.3.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.3.1
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.5
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.9
bokeh==2.3.3
branca==0.6.0
bs4==0.0.1
CacheControl==0.12.11
cachetools==5.2.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.8
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.3
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.0
cvxpy==1.2.2
cycler==0.11.0
cymem==2.0.7
Cython==0.29.32
daft==0.0.4
dask==2022.2.1
datascience==0.17.5
db-dtypes==1.0.5
debugpy==1.0.0
decorator==4.4.2
d

In [ ]:
!pip install torchtext==0.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torchtext-0.8.1-cp38-cp38-manylinux1_x86_64.whl (7.0 MB)
  Using cached torch-1.7.1-cp38-cp38-manylinux1_x86_64.whl (776.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import torchtext

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [ ]:
batch_size = 30
max_length = 256

TEXT = torchtext.data.Field(
    lower=True, include_lengths=False, batch_first=True
)
print(TEXT)
LABEL = torchtext.data.Field(sequential=False)
print(LABEL)

train_txt, test_txt = torchtext.datasets.IMDB.splits(TEXT, LABEL)
print(train_txt)
print(test_txt)

TEXT.build_vocab(
    train_txt,
    vectors=torchtext.vocab.GloVe(name="6B", dim=50, max_vectors=50_000),
    max_size=50_000,
)

# LABEL.build_vocab(train_txt)

# train_iter, test_iter = torchtext.data.BucketIterator.splits(
#     (train_txt, test_txt),
#     batch_size=batch_size,
# )

AttributeError: ignored

In [ ]:
write_line_header_1 = f'word,pos,bio,stemmed,pos_back1,word_back1,stemmed_back1,bio_back1,pos_back2,word_back2,stemmed_back2,bio_back2,pos_plus1,word_plus1,stemmed_plus1,bio_plus1,pos_plus2,word_plus2,stemmed_plus2,bio_plus2,token_count,sentence_id,is_support,is_pred,part_type,is_arg1'
featues_list_1 = write_line_header_1.split(',')

def convert_to_featureset(file):
    df = pd.read_csv(file)
    sentence = list()
    label = list()
    answers = list()

    train_unique_words_caps = set()
    train_unique_words_ncaps = set()

    for index, row in df.iterrows():
      word = row['word']
      train_unique_words_caps.add(word)
      train_unique_words_ncaps.add(word.lower())
      sentence.append(word)
      # train_uniqe
      if(row['is_pred']):
        label.append(1)
      elif(row['is_arg1']):
        label.append(2)
      else:
        label.append(0)

      if(row['word_plus1'] == 'END'):
        feature_row = [' '.join(sentence), label]
        # max_sentence_length = max(max_sentence_length, len(label))
        answers.append(feature_row)
        sentence = list()
        label = list()
      
    df_sentence = pd.DataFrame(answers, columns = ['text', 'label'])
    print('caps:', len(train_unique_words_caps), 'ncaps:', len(train_unique_words_ncaps))
    return df_sentence


    # df_new = df.copy()
    # for i in featues_list_1:
    #     df_new[i] = (df_new[i].astype('category')).cat.codes

    # for column in df_new.columns:
    #   df_new[column] = df_new[column]  / df_new[column].abs().max()

    # #print(df.head())
    # return df, df_new

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd  /content/drive/MyDrive/NLP/

/content/drive/MyDrive/NLP


In [ ]:
#/content/drive/MyDrive/NLP/New/part_dataset_dev.csv
train_linear_reg_df = convert_to_featureset('part_dataset_train.csv')
dev_linear_reg_df = convert_to_featureset('part_dataset_dev.csv')
test_linear_reg_df = convert_to_featureset('part_dataset_test.csv')

caps: 22125 ncaps: 20241
caps: 2775 ncaps: 2636
caps: 3731 ncaps: 3502


In [ ]:
train_linear_reg_df.head()

,text,label
0,But about 25 % of the insiders COMMA according...,"[0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,The bond fund will invest in high-grade or med...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,The position of the United States COMMA which ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, ..."
3,The first hybrid corn seeds produced using thi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Mr. Leemans said the hybrid rapeseeds created ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [ ]:
from torchtext.data import Field
import torchtext
from torchtext.data import Dataset, Example

In [ ]:
class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [ ]:
def custom_tokenizer(text):
    return text.split(' ')

In [ ]:
def create_dataset(df, train_vocab = None):
  text_field = Field(
      sequential=True,
      #tokenize='basic_english', 
      tokenize=custom_tokenizer,
      fix_length=None,
      lower=False, 
      batch_first=True
  )
  label_field = Field(sequential=True, use_vocab=False, pad_token=0, batch_first=True)

  # sadly have to apply preprocess manually
  #preprocessed_text = df['text'].apply(
  #    lambda x: text_field.preprocess(x)
  #)

  print(df['text'][0:10])
  preprocessed_text = df['text'].apply(
        lambda y: y.split(' ')
  )
  print(preprocessed_text[0:10])
  # load fastext simple embedding with 300d
  text_field.build_vocab(
      preprocessed_text, 
      vectors='fasttext.simple.300d'
  )
  # get the vocab instance
  if(train_vocab):
    vocab = train_vocab
  else:
    vocab = text_field.vocab

  
  # df['label'] = df['label'].apply(lambda y: [[ans_map[lab] for lab in y.split(' ')]])
  #ltoi = {l: i for i, l in enumerate(df['label'].unique())}
  #df['label'] = df['label'].apply(lambda y: ltoi[y])

  batch_size = 30
  max_length = 256


  dataset = DataFrameDataset(
      df=df, 
      fields=(
          ('text', text_field),
          ('label', label_field)
      )
  )
  return dataset, vocab

# train_iter, test_iter = torchtext.data.BucketIterator.splits(
#     (train_dataset, train_dataset),
#     batch_size=16,
# )
# for idx, batch in enumerate(iter(train_iter)):
# 	import pdb; pdb.set_trace()
# #TEXT = torchtext.data.Field(
#    lower=True, include_lengths=False, batch_first=True
#)


In [ ]:
# [ans_map[lab] for lab in train_linear_reg_df['label'][0].split(' ')]

In [ ]:
train_dataset, train_vocab = create_dataset(train_linear_reg_df)
dev_dataset, _ = create_dataset(dev_linear_reg_df, train_vocab)
test_dataset, _ = create_dataset(test_linear_reg_df, train_vocab)

/usr/local/lib/python3.8/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


0    But about 25 % of the insiders COMMA according...
1    The bond fund will invest in high-grade or med...
2    The position of the United States COMMA which ...
3    The first hybrid corn seeds produced using thi...
4    Mr. Leemans said the hybrid rapeseeds created ...
5    Mr. Leemans said the hybrid rapeseeds created ...
6    Alltel Corp. said it will acquire the 55 % of ...
7    Alltel holds 45 % of the franchise COMMA which...
8    Canada 's gross domestic product rose an infla...
9    The August GDP was up 2.4 % from its year-earl...
Name: text, dtype: object
0    [But, about, 25, %, of, the, insiders, COMMA, ...
1    [The, bond, fund, will, invest, in, high-grade...
2    [The, position, of, the, United, States, COMMA...
3    [The, first, hybrid, corn, seeds, produced, us...
4    [Mr., Leemans, said, the, hybrid, rapeseeds, c...
5    [Mr., Leemans, said, the, hybrid, rapeseeds, c...
6    [Alltel, Corp., said, it, will, acquire, the, ...
7    [Alltel, holds, 45, %, of, the, fr

/usr/local/lib/python3.8/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


0    The consensus view expects a 0.4 % increase in...
1    Food prices are expected to be unchanged COMMA...
2    The new financing structure is still a very-hi...
3    GM 's domestic car sales dropped 24.3 % and it...
4    GM 's domestic car sales dropped 24.3 % and it...
5    Cadillac posted a 3.2 % increase despite new c...
6    Ford Motor Co. had a 1.8 % drop in domestic ca...
7    Ford Motor Co. had a 1.8 % drop in domestic ca...
8    Chrysler Corp. had a 7.5 % drop in car sales C...
9    However COMMA sales of trucks COMMA including ...
Name: text, dtype: object
0    [The, consensus, view, expects, a, 0.4, %, inc...
1    [Food, prices, are, expected, to, be, unchange...
2    [The, new, financing, structure, is, still, a,...
3    [GM, 's, domestic, car, sales, dropped, 24.3, ...
4    [GM, 's, domestic, car, sales, dropped, 24.3, ...
5    [Cadillac, posted, a, 3.2, %, increase, despit...
6    [Ford, Motor, Co., had, a, 1.8, %, drop, in, d...
7    [Ford, Motor, Co., had, a, 1.8, %,

In [ ]:
print(train_dataset[0])

In [21]:
train_iter = torchtext.data.BucketIterator.splits(
    (train_dataset),
    batch_size=16,
   # sort = False
)

AttributeError: ignored

In [ ]:
dev_iter, test_iter = torchtext.data.BucketIterator.splits(
    (dev_dataset,test_dataset ),
    batch_size=1,
)

In [ ]:
for idx, batch in enumerate(iter(train_iter)):
  print(batch.label)
  print(batch.text.shape, batch.label.shape)
  if idx > 10: 
    break

tensor([[0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0,

/usr/local/lib/python3.8/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
import math

import torch
import torch.nn as nn

#import torchtext

class PositionalEncoding(nn.Module):
    """
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

class Net(nn.Module):
    """
    Text classifier based on a pytorch TransformerEncoder.
    """

    def __init__(
        self,
        embeddings,
        nhead=8,
        dim_feedforward=2048,
        num_layers=6,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1,
    ):

        super().__init__()

        vocab_size, d_model = embeddings.size()
        assert d_model % nhead == 0, "nheads must divide evenly into d_model"

        self.emb = nn.Embedding.from_pretrained(embeddings, freeze=False)

        self.pos_encoder = PositionalEncoding(
            d_model=d_model,
            dropout=dropout,
            vocab_size=vocab_size,
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.classifier = nn.Linear(d_model, 3)
        self.d_model = d_model
    def forward(self, x):
        x = self.emb(x) * math.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        #x = x.mean(dim=1)
        x = self.classifier(x)

        return x


In [ ]:
# from net import Net 
# import torch
# import torch.nn as nn
# from dataset import vocab, train_iter

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
epochs = 5
model = Net(
    train_vocab.vectors,
    nhead=5,  # the number of heads in the multiheadattention models
    dim_feedforward=50,  # the dimension of the feedforward network model in nn.TransformerEncoder
    num_layers=6,
    dropout=0.0,
    classifier_dropout=0.0,
).to(device)

criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0,2.0,2.0]))

lr = 1e-4
optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=lr,  betas=(0.9, 0.999), weight_decay=0.00001
)

torch.manual_seed(0)
eval_frequency = 10
for epoch in range(epochs):
    model.train()
    start = time.time()
    print(f"{epoch}")
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for idx, batch in enumerate(iter(train_iter)):
        
        #print('batch:', batch.text.shape, batch.label.shape)
        optimizer.zero_grad()
        predictions = model(batch.text.to(device))
        #print('predictions:', predictions.shape)
        predictions = predictions.reshape(predictions.shape[0] * predictions.shape[1], 3)
        # print(predictions.shape)
        labels = batch.label.to(device)
        #print('labels:', labels.shape)
        labels = labels.reshape(labels.shape[0] * labels.shape[1])
        
        loss = criterion(predictions, labels)
        # print('loss:', loss)
        correct = predictions.argmax(axis=-1) == labels
        correct_1 = correct[torch.where(labels==1)]
        correct_2 = correct[torch.where(labels==2)]
        if idx%100 == 0:
          print('idx:', idx, 'loss:', loss, 'correct1:', torch.sum(correct_1), 'correct2:', torch.sum(correct_2) )
        # print('loss:', loss, 'correct1:', torch.sum(correct_1), 'correct2:', torch.sum(correct_2) )
        # print('correct:', correct)
        # print('torch:', torch.sum(correct), correct.shape)
        #import pdb; pdb.set_trace()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
    end = time.time()
    print('time for epoch:', str(end-start))
    checkpoint = { 
    'epoch': epoch,
    'model': model,
    'optimizer': optimizer
    }
    torch.save(checkpoint, f'checkpoint{epoch}.pth')
  

cpu
0
idx: 0 loss: tensor(0.6547, grad_fn=<NllLossBackward>) correct1: tensor(0) correct2: tensor(5)


/usr/local/lib/python3.8/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


idx: 100 loss: tensor(0.1474, grad_fn=<NllLossBackward>) correct1: tensor(13) correct2: tensor(1)
idx: 200 loss: tensor(0.1118, grad_fn=<NllLossBackward>) correct1: tensor(16) correct2: tensor(5)
idx: 300 loss: tensor(0.1101, grad_fn=<NllLossBackward>) correct1: tensor(13) correct2: tensor(1)
idx: 400 loss: tensor(0.0774, grad_fn=<NllLossBackward>) correct1: tensor(16) correct2: tensor(2)
idx: 500 loss: tensor(0.1348, grad_fn=<NllLossBackward>) correct1: tensor(14) correct2: tensor(2)
idx: 600 loss: tensor(0.1335, grad_fn=<NllLossBackward>) correct1: tensor(13) correct2: tensor(0)
time for epoch: 295.2688958644867
1
idx: 0 loss: tensor(0.0977, grad_fn=<NllLossBackward>) correct1: tensor(14) correct2: tensor(3)
idx: 100 loss: tensor(0.1170, grad_fn=<NllLossBackward>) correct1: tensor(15) correct2: tensor(5)
idx: 200 loss: tensor(0.1179, grad_fn=<NllLossBackward>) correct1: tensor(14) correct2: tensor(4)
idx: 300 loss: tensor(0.1218, grad_fn=<NllLossBackward>) correct1: tensor(15) correc

In [ ]:
checkpoint = { 
    'epoch': 5,
    'model': model,
    'optimizer': optimizer
    }
torch.save(checkpoint, f'checkpoint5.pth')

In [ ]:
checkpoint = torch.load('checkpoint5.pth')

In [ ]:
 for epoch in range(1):
    if epoch%eval_frequency == 0:
      model.eval()
      for eval_idx, eval_batch in enumerate(iter(dev_iter)):
        predictions = model(eval_batch.text.to(device))
        predictions = predictions.reshape(predictions.shape[0] * predictions.shape[1], 3)
        labels = eval_batch.label.to(device)
        #print('labels:', labels.shape)
        labels = labels.reshape(labels.shape[0] * labels.shape[1])
        # print('loss:', loss)
        correct = predictions.argmax(axis=-1) == labels
        correct_1 = correct[torch.where(labels==1)]
        correct_2 = correct[torch.where(labels==2)]
        # print('loss:', loss, 'correct1:', torch.sum(correct_1), 'correct2:', torch.sum(correct_2) )